In [2]:
# Step 1: Install dependencies
# pip install langchain langchain_community langchain-google-genai chromadb

import os
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA

# -------------------------------
# Step 2: Prepare Documents & Vector Store
# -------------------------------

# Load documents (replace with your file path)
loader = TextLoader("solar_system.txt")
docs = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

# Embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Store in Chroma
vectorstore = Chroma.from_documents(documents, embedding=embeddings)

# Create Retriever
retriever = vectorstore.as_retriever()

# -------------------------------
# Step 3: Define RAG Tool
# -------------------------------

rag_chain = RetrievalQA.from_chain_type(
    llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0),
    retriever=retriever,
    chain_type="stuff"
)

tools = [
    Tool(
        name="Knowledge Base",
        func=rag_chain.run,
        description="Use this tool to answer questions from the uploaded documents."
    )
]

# -------------------------------
# Step 4: Initialize Agent
# -------------------------------

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # correct arg name
    verbose=True
)

# -------------------------------
# Step 5: Run Agent
# -------------------------------

query = "Summarize the document for me."
response = agent.run(query)   # use run() instead of invoke()

print("Agent Response:\n", response)


C:\Users\acer\AppData\Local\Temp\ipykernel_15940\2902224754.py:58: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\acer\AppData\Local\Temp\ipykernel_15940\2902224754.py:70: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)   # use run() instead of 



> Entering new AgentExecutor chain...
Thought: I need to use the Knowledge Base tool to access and summarize the provided document.  I don't have access to any documents yet, so I need that information first.

Action: Knowledge Base

Action Input:  Please provide the document you wish me to summarize.
Observation: The provided text is about the Solar System.  It describes its composition (Sun, planets, moons, asteroids, comets, meteoroids, dwarf planets), the relative sizes and locations of its components, and the characteristics of each component type.
Thought:Thought: I now have the necessary information from the Observation. I can now summarize the document.

Final Answer: The document describes our Solar System, outlining its major components: the Sun, planets (both inner rocky and outer gas giants), moons, asteroids, comets, meteoroids, and dwarf planets.  It details the relative sizes and positions of these components and provides a general overview of their characteristics.

>